In [5]:
#https://www.kaggle.com/c/plant-seedlings-classification
from __future__ import division, print_function


In [6]:
sample_or_prod = True #True = sample, False = production

train_valid_fract = 0.3
sample_fract = 0.2

kg download -u user_email -p password -c plant-seedlings-classification  
unzip sample_submission.csv.zip  
unzip test.zip  
unzip train.zip  


In [43]:
import os
from datetime import datetime
from glob import glob
from random import shuffle
from shutil import copyfile


In [24]:
HOMEPATH = "/home/ubuntu/fastai/"
DATA_PATH = HOMEPATH + "data/Kaggle_seedlings/"
print("HOMEPATH:", HOMEPATH)
print("DATA_PATH:", DATA_PATH)


os.chdir(DATA_PATH)
print (os.getcwd())
!tree -d

HOMEPATH: /home/ubuntu/fastai/
DATA_PATH: /home/ubuntu/fastai/data/Kaggle_seedlings/
/home/ubuntu/fastai/data/Kaggle_seedlings
.
├── test
└── train
    ├── Black-grass
    ├── Charlock
    ├── Cleavers
    ├── Common Chickweed
    ├── Common wheat
    ├── Fat Hen
    ├── Loose Silky-bent
    ├── Maize
    ├── Scentless Mayweed
    ├── Shepherds Purse
    ├── Small-flowered Cranesbill
    └── Sugar beet

14 directories


In [25]:
MODEL_PATH = DATA_PATH+"models/"
RESULTS_PATH = DATA_PATH+"results/"


SAMPLE_DATA_PATH = DATA_PATH + "sample/"#choose this for testing or above for production
SAMPLE_TEST_PATH = SAMPLE_DATA_PATH+"test/"
SAMPLE_TRAIN_PATH = SAMPLE_DATA_PATH + "train/"
SAMPLE_VALID_PATH = SAMPLE_DATA_PATH + "valid/"


TEST_PATH = DATA_PATH+"test/"
TRAIN_PATH = DATA_PATH + "train/"
VALID_PATH = DATA_PATH + "valid/"


if sample_or_prod:
    WORKING_DATA  = SAMPLE_DATA_PATH
    WORKING_TEST  = SAMPLE_TEST_PATH
    WORKING_TRAIN = SAMPLE_TRAIN_PATH
    WORKING_VALID = SAMPLE_VALID_PATH
    s_or_p = "_sample_"
else:
    WORKING_DATA  = DATA_PATH
    WORKING_TEST  = TEST_PATH
    WORKING_TRAIN = TRAIN_PATH
    WORKING_VALID = VALID_PATH
    s_or_p = "_prod_"

    
print ("WORKING_DATA:", WORKING_DATA)
print ("WORKING_TEST:", WORKING_TEST)
print ("WORKING_TRAIN:", WORKING_TRAIN)
print ("WORKING_VALID:", WORKING_VALID)
print ("s_or_p:", s_or_p)

WORKING_DATA: /home/ubuntu/fastai/data/Kaggle_seedlings/sample/
WORKING_TEST: /home/ubuntu/fastai/data/Kaggle_seedlings/sample/test/
WORKING_TRAIN: /home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/
WORKING_VALID: /home/ubuntu/fastai/data/Kaggle_seedlings/sample/valid/
s_or_p: _sample_


In [26]:
def dirFileList(dir_path):
    return [name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))]


In [27]:
#check if dir exists, create if not already.
def makeNewDir(newDirPath):
    if not os.path.exists(newDirPath): 
        print("directory ", newDirPath, " did not exist, creating:")
        os.mkdir(newDirPath)
    else:
        print("directory ", newDirPath, "already existed. filecount = ", len(dirFileList(newDirPath)))

In [28]:
#nb: do not change order 
#NBB: must have trailing slash

CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
for i in range(len(CATEGORIES)):
    CATEGORIES[i] = CATEGORIES[i] + "/"
print (CATEGORIES)


['Black-grass/', 'Charlock/', 'Cleavers/', 'Common Chickweed/', 'Common wheat/', 'Fat Hen/', 'Loose Silky-bent/', 'Maize/', 'Scentless Mayweed/', 'Shepherds Purse/', 'Small-flowered Cranesbill/', 'Sugar beet/']


In [29]:
UNKNOWN = 'unknown/'

#NB: train and test are created by unzipping the data.
makeNewDir(SAMPLE_DATA_PATH)
makeNewDir(SAMPLE_TEST_PATH)
makeNewDir(SAMPLE_TEST_PATH+UNKNOWN)
makeNewDir(SAMPLE_TRAIN_PATH)
makeNewDir(SAMPLE_VALID_PATH)

makeNewDir(MODEL_PATH)
makeNewDir(RESULTS_PATH)

makeNewDir(TEST_PATH)
makeNewDir(TEST_PATH+UNKNOWN)
makeNewDir(TRAIN_PATH)
makeNewDir(VALID_PATH)

DATA_DIR_LIST = [DATA_PATH, SAMPLE_DATA_PATH, SAMPLE_TEST_PATH, SAMPLE_TEST_PATH+UNKNOWN, SAMPLE_TRAIN_PATH, 
                 SAMPLE_VALID_PATH, TEST_PATH, TEST_PATH+UNKNOWN, TRAIN_PATH, VALID_PATH, MODEL_PATH, RESULTS_PATH]

print ("DATA_DIR_LIST:", DATA_DIR_LIST)


for category in CATEGORIES:
    makeNewDir(TRAIN_PATH+category)
    makeNewDir(VALID_PATH+category)
    makeNewDir(SAMPLE_TRAIN_PATH+category)
    makeNewDir(SAMPLE_VALID_PATH+category)
    #
    DATA_DIR_LIST.append(TRAIN_PATH+category)
    DATA_DIR_LIST.append(VALID_PATH+category)
    DATA_DIR_LIST.append(SAMPLE_TRAIN_PATH+category)
    DATA_DIR_LIST.append(SAMPLE_VALID_PATH+category)


directory  /home/ubuntu/fastai/data/Kaggle_seedlings/sample/  did not exist, creating:
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/sample/test/  did not exist, creating:
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/sample/test/unknown/  did not exist, creating:
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/  did not exist, creating:
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/sample/valid/  did not exist, creating:
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/models/  did not exist, creating:
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/results/  did not exist, creating:
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/test/ already existed. filecount =  794
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/test/unknown/  did not exist, creating:
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/train/ already existed. filecount =  0
directory  /home/ubuntu/fastai/data/Kaggle_seedlings/valid/  did not exist, creati

In [30]:
def listDirsFileCount(DATA_DIR_LIST):
    DATA_DIR_LIST = sorted(DATA_DIR_LIST)
    for dir_ in DATA_DIR_LIST:
        print (dir_, len(dirFileList(dir_)))
        


In [31]:
listDirsFileCount(DATA_DIR_LIST)

/home/ubuntu/fastai/data/Kaggle_seedlings/ 4
/home/ubuntu/fastai/data/Kaggle_seedlings/models/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/results/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/test/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/test/unknown/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Black-grass/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Charlock/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Cleavers/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Common Chickweed/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Common wheat/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Fat Hen/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Loose Silky-bent/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Maize/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Scentles

In [32]:
os.chdir(DATA_PATH)
print (os.getcwd())
!tree -d

/home/ubuntu/fastai/data/Kaggle_seedlings
.
├── models
├── results
├── sample
│   ├── test
│   │   └── unknown
│   ├── train
│   │   ├── Black-grass
│   │   ├── Charlock
│   │   ├── Cleavers
│   │   ├── Common Chickweed
│   │   ├── Common wheat
│   │   ├── Fat Hen
│   │   ├── Loose Silky-bent
│   │   ├── Maize
│   │   ├── Scentless Mayweed
│   │   ├── Shepherds Purse
│   │   ├── Small-flowered Cranesbill
│   │   └── Sugar beet
│   └── valid
│       ├── Black-grass
│       ├── Charlock
│       ├── Cleavers
│       ├── Common Chickweed
│       ├── Common wheat
│       ├── Fat Hen
│       ├── Loose Silky-bent
│       ├── Maize
│       ├── Scentless Mayweed
│       ├── Shepherds Purse
│       ├── Small-flowered Cranesbill
│       └── Sugar beet
├── test
│   └── unknown
├── train
│   ├── Black-grass
│   ├── Charlock
│   ├── Cleavers
│   ├── Common Chickweed
│   ├── Common wheat
│   ├── Fat Hen
│   ├── Loose Silky-bent
│   ├── Maize
│   ├── Scentless Mayweed
│   ├── Shepherds Purse
│   ├── S

In [33]:
def moveCopyFiles(fileList, orig, dest, moveCopy):
    #if moveCopy=True then move files
    #if moveCopy=False then copy files
    success = 0
    error = 0
    count = 0
    for f in fileList:
        filename = orig+f
        destFilename = dest+f
        try:
            #print "moving from ", filename+" to ", destFilename
            if moveCopy:
                os.rename(filename, destFilename)
            else:
                copyfile(filename, destFilename)
            success += 1
        except:
            print("error moving :", filename)
            error += 1
        count += 1
        #break
    print ("success:", success)
    print ("error:", error)
    print ("count:", count)


In [36]:
#move files from test to test/unknown
os.chdir(TEST_PATH)
print ("os.getcwd:", os.getcwd())
files = glob('*.png')
print ("len(files):", len(files))

print ("# of files in dir :", TEST_PATH, ":", len(dirFileList(TEST_PATH)))
moveCopyFiles(files, TEST_PATH, TEST_PATH+UNKNOWN, True)
print ("# of files in dir ", TEST_PATH+UNKNOWN, ":", len(dirFileList(TEST_PATH+UNKNOWN)))

os.getcwd: /home/ubuntu/fastai/data/Kaggle_seedlings/test
len(files): 794
# of files in dir : /home/ubuntu/fastai/data/Kaggle_seedlings/test/ : 794
success: 794
error: 0
count: 794
# of files in dir  /home/ubuntu/fastai/data/Kaggle_seedlings/test/unknown/ : 794


In [41]:
def splitMoveCopyFiles(fraction, origDir, destDir, moveOrCopy):
    #if moveCopy=True then move files
    #if moveCopy=False then copy files
    #print (categories[0])
    fileList = dirFileList(origDir)
    shuffle(fileList)
    print ("Number of files in original directory :", len(fileList))
    num_files_to_move = int(len(fileList) * fraction)
    print ("num_files_to_move:", num_files_to_move)
    files = fileList[0:num_files_to_move]
    moveCopyFiles(files, origDir, destDir, moveOrCopy)


In [46]:
#get list of files, randomly subset from list, move to valid. repeat for each category in categories list.
#NB: test if destination directory is empty first to avoid repeating moves.

for category in CATEGORIES:
    moveList = dirFileList(TRAIN_PATH+category)
    print (len(moveList), train_valid_fract, TRAIN_PATH+category, VALID_PATH+category)
    #check if destination directory is empty to avoid repeat moves.
    destFileList = dirFileList(VALID_PATH+category)
    if len(destFileList)==0:
        splitMoveCopyFiles(train_valid_fract, TRAIN_PATH+category, VALID_PATH+category, True)
        # now select and _copy_ random files from TRAIN_PATH+category to SAMPLE_TRAIN_PATH+category
        splitMoveCopyFiles(sample_fract, TRAIN_PATH+category, SAMPLE_TRAIN_PATH+category, False)
        # now select and _copy_ random files from VALID_PATH+category to SAMPLE_VALID_PATH+category
        splitMoveCopyFiles(sample_fract, VALID_PATH+category, SAMPLE_VALID_PATH+category, False)
    else:
        print ("files already in destination directory, do NOT move.")

    # now select and _copy_ random files from TRAIN_PATH+category to SAMPLE_TRAIN_PATH+category
    splitMoveCopyFiles(sample_fract, TRAIN_PATH+category, SAMPLE_TRAIN_PATH+category, False)
    # now select and _copy_ random files from VALID_PATH+category to SAMPLE_VALID_PATH+category
    splitMoveCopyFiles(sample_fract, VALID_PATH+category, SAMPLE_VALID_PATH+category, False)


263 0.3 /home/ubuntu/fastai/data/Kaggle_seedlings/train/Black-grass/ /home/ubuntu/fastai/data/Kaggle_seedlings/valid/Black-grass/
Number of files in original directory : 263
num_files_to_move: 78
success: 78
error: 0
count: 78
Number of files in original directory : 185
num_files_to_move: 37
success: 37
error: 0
count: 37
Number of files in original directory : 78
num_files_to_move: 15
success: 15
error: 0
count: 15
Number of files in original directory : 185
num_files_to_move: 37
success: 37
error: 0
count: 37
Number of files in original directory : 78
num_files_to_move: 15
success: 15
error: 0
count: 15
390 0.3 /home/ubuntu/fastai/data/Kaggle_seedlings/train/Charlock/ /home/ubuntu/fastai/data/Kaggle_seedlings/valid/Charlock/
Number of files in original directory : 390
num_files_to_move: 117
success: 117
error: 0
count: 117
Number of files in original directory : 273
num_files_to_move: 54
success: 54
error: 0
count: 54
Number of files in original directory : 117
num_files_to_move: 23


In [48]:
splitMoveCopyFiles(sample_fract, TEST_PATH+UNKNOWN, SAMPLE_TEST_PATH+UNKNOWN, False)

Number of files in original directory : 794
num_files_to_move: 158
success: 158
error: 0
count: 158


In [49]:
listDirsFileCount(DATA_DIR_LIST)

/home/ubuntu/fastai/data/Kaggle_seedlings/ 4
/home/ubuntu/fastai/data/Kaggle_seedlings/models/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/results/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/test/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/test/unknown/ 158
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/ 0
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Black-grass/ 65
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Charlock/ 98
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Cleavers/ 76
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Common Chickweed/ 154
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Common wheat/ 58
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Fat Hen/ 123
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Loose Silky-bent/ 166
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/train/Maize/ 58
/home/ubuntu/fastai/data/Kaggle_seedlings/sample/t

In [50]:
os.chdir(DATA_PATH)
print (os.getcwd())
!tree -d

/home/ubuntu/fastai/data/Kaggle_seedlings
.
├── models
├── results
├── sample
│   ├── test
│   │   └── unknown
│   ├── train
│   │   ├── Black-grass
│   │   ├── Charlock
│   │   ├── Cleavers
│   │   ├── Common Chickweed
│   │   ├── Common wheat
│   │   ├── Fat Hen
│   │   ├── Loose Silky-bent
│   │   ├── Maize
│   │   ├── Scentless Mayweed
│   │   ├── Shepherds Purse
│   │   ├── Small-flowered Cranesbill
│   │   └── Sugar beet
│   └── valid
│       ├── Black-grass
│       ├── Charlock
│       ├── Cleavers
│       ├── Common Chickweed
│       ├── Common wheat
│       ├── Fat Hen
│       ├── Loose Silky-bent
│       ├── Maize
│       ├── Scentless Mayweed
│       ├── Shepherds Purse
│       ├── Small-flowered Cranesbill
│       └── Sugar beet
├── test
│   └── unknown
├── train
│   ├── Black-grass
│   ├── Charlock
│   ├── Cleavers
│   ├── Common Chickweed
│   ├── Common wheat
│   ├── Fat Hen
│   ├── Loose Silky-bent
│   ├── Maize
│   ├── Scentless Mayweed
│   ├── Shepherds Purse
│   ├── S